In [1]:
import pandas as pd
import random
import numpy as np

#### Load Data

In [2]:
dfTripData = pd.read_csv("location_data.csv")

#### Script Parameters

In [3]:
intNumHouseNodes = 5
intNumCommNodes = 2
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh

#### Functions

In [23]:
# create residential nodes

def createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData):
    #initialize data frame to hold master
    dfNodesTrips = pd.DataFrame(columns=['TripId','ResNode','CommNode'])  

    for intNode in range(intNumHouseNodes):

        # find how many vehicles will be at that residential node
        intNodeNumCars = int(random.gauss(intAvgNumCarsNode,intSdNumCarsNode))
        # assign cars to that node
        lsCarIds = [] # container to hold trip id for that node
        lsCommNode = [] # container to hold where the commerical node 
        lsResNode = [] # container to hold the res node
        for intCar in range(intNodeNumCars):

            # find a car from the data frame
            ind = random.randint(0,intProfiles)
            lsCarIds.append(dfTripData['ID'].iloc[ind]) # add to car id container

            # save res node
            lsResNode.append(intNode)

            # find which commercial node that car goes to
            lsCommNode.append(random.randint(1,intNumCommNodes))

        # create data frame and add to master
        dfNode = pd.DataFrame(list(zip(lsCarIds,lsResNode,lsCommNode)), \
                              columns=['TripId','ResNode','CommNode'])

        dfNodesTrips = dfNodesTrips.append(dfNode)

    dfNodesTrips.reset_index(inplace=True, drop=True)
    
    return dfNodesTrips

    

In [61]:
def getSOCconstraints(dfNodesTrips, fltCarEff,fltBatteryCap):

    intNumProfs = dfNodesTrips.shape[0]
    arrSOCrequirements = np.zeros((intNumProfs, 96))

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # and then find the average miles travelled each interval
        fltAvgMileInterval = fltMiles / intAway

        intStart = 0 
        while intStart < 96 and 'A' in lsLocations[intStart:]:

            # Find the first first time we start moving
            intStart = intStart + lsLocations[intStart:].index('A') 

            # from there find when we arrive at a destination
            if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
                intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
            elif 'W' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('W')
            elif 'H' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('H')
            else: # we are going to delete that profile cause something is wrong
                print(lsLocations[intStart:])
                dfNodesTrips = dfNodesTrips.drop([ind])

            # intervals driving for trip is then (intStart+intEnd +1) - intStart = intEnd =1
            # then we can find miles travelled for first trip 
            fltMilesTraveled = fltAvgMileInterval * (intEnd)
            fltKwhRequirement = fltMilesTraveled / fltCarEff # miles / (miles/kWh)

            # now find SOC required
            fltInitialSoc = fltKwhRequirement / fltBatteryCap

            # so we need to to have the initial SOC at the interval before this time:
            arrSOCrequirements[ind,intStart-1] = fltInitialSOC

            intStart += intEnd # increment the starting point for finding the next trip

    return arrSOCrequirements


#### Execute

In [62]:
# call functions to get Nodes Dataframe
dfNodesTrips = createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

# now call function to get the SOC constraints array
arrSOCconstraint = getSOCconstraints(dfNodesTrips, fltCarEff,fltBatteryCap)